In [ ]:
import json

with open('_CBDA_Ship_1_ETL_Preprocessing.json', 'r') as f:
    r = f.readline()
    
r = r.encode().decode('utf-8-sig')

j = json.loads(r)

for cell in j['paragraphs']:
    cell_text = cell['text']
    print(cell_text, end='\n\n')

In [ ]:
# %pyspark
print(spark.version)
from pyspark.sql.functions import *
from pyspark.sql.types import *

# %pyspark
spark.sql("use d4sa_us_disc")

# %pyspark

# Apply filters to discovery table so that the numbers align with publish tables that MDLZ stakeholders use for reporting / analysis
raw_shipments_curr = spark.table("d4sa_us_disc.fact_spark56_daily_int_00_001").\
filter("D_BIC_ZMOD_IND = 'ST'").\
filter("D_BIC_ZPHIER03 != ''").\
filter("D_VERSION == '000'").\
filter("(D_BILL_DATE != '00000000') OR (D_BILL_DATE == '00000000' AND D_BIC_ZINFOPROV == 'ZSCNNC53')").\
filter("D_BIC_ZCAHIER01 != 'US1000012'").\
filter("D_ZCAHIER01_T != ''").\
filter("D_BIC_ZG_AVV004 != 0")
# Filter for only 2020 data from this table since 2019 data (esp earlier weeks) seem incomplete
raw_shipments_curr = raw_shipments_curr.filter("D_FISCPER IN ('2020001','2020002','2020003','2020004','2020005','2020006','2020007','2020008','2020009','2020010','2020011','2020012')")

raw_shipments_hist = spark.table("d4sa_us_disc.fact_spark56_daily_hist_int_00_001").\
filter("D_BIC_ZMOD_IND = 'ST'").\
filter("D_BIC_ZPHIER03 != ''").\
filter("D_VERSION == '000'").\
filter("(D_BILL_DATE != '00000000') OR (D_BILL_DATE == '00000000' AND D_BIC_ZINFOPROV == 'ZSCNNC53')").\
filter("D_BIC_ZCAHIER01 != 'US1000012'").\
filter("D_ZCAHIER01_T != ''").\
filter("D_BIC_ZG_AVV004 != 0")
raw_shipments_hist = raw_shipments_hist.filter("D_FISCPER IN ('2019001','2019002','2019003','2019004','2019005','2019006','2019007','2019008','2019009','2019010','2019011','2019012')")

# Append the 2020 curr and 2019 hist DFs
raw_shipments_hist = raw_shipments_hist.select(*raw_shipments_curr.columns)
raw_shipments = raw_shipments_curr.union(raw_shipments_hist)

raw_shipments.createOrReplaceTempView("fact")

# DO NOT USE - filter("D_BIC_ZCGHIER02 NOT IN ('G20000003','G20000008')")

# %pyspark

# Read cust hiererachy data
cust_hier = spark.table("hierarchies_customer_curr_hierarchy_int_00_001")

# Rename Dist channel ID column
cust_hier = cust_hier.withColumnRenamed("distribution_channel_id", "distribution_channel_id_cust")

# pad 0s to store_id col
cust_hier = cust_hier.withColumn("store_id" , lpad(cust_hier['store_id'],10,'0').alias('store_id'))
cust_hier.createOrReplaceTempView("hierarchies_customer")

# Filter for gc_area_id not null - Use an inner join with fact to filter out there as well
#cust_hier = cust_hier.filter("gc_area_id IS NOT NULL")

z.show(cust_hier)

# %pyspark

prod_hier = spark.table("hierarchies_product_curr_hierarchy_int_00_001")
# Rename Dist channel ID column
prod_hier = prod_hier.withColumnRenamed("distribution_channel_id", "distribution_channel_id_prod")
# pad 0s to SKU_Id col
prod_hier = prod_hier.withColumn("sku_id" , lpad(prod_hier['sku_id'],18,'0').alias('sku_id'))

prod_hier.createOrReplaceTempView("hierarchies_product")

z.show(prod_hier.limit(10))

# %pyspark
query = """
select g.date0, g.fiscper, g.promoted_product_group, g.promoted_product_group_desc, g.product_family_desc, g.product_segment_name, g.product_category_name, g.management_grouping_desc, g.state, g.ac_scbm_id, g.ac_scbm_desc, g.bic_zdistr_ch, g.state, sum(bic_zg_avv004) as gross_sales, sum(bic_znt_wt_lb) as sales_pounds
from 
(select *
 from (select D_BIC_ZFISCDAY as date0, D_FISCPER as fiscper, D_CUSTOMER as customer, D_MATERIAL as material, D_BIC_ZDISTR_CH as bic_zdistr_ch, D_REGION as state, D_BIC_ZG_AVV004 as bic_zg_avv004, D_BIC_ZNT_WT_LB as bic_znt_wt_lb
      from fact
      where D_FISCYEAR IN ('2020','2019')) a
 left join hierarchies_customer b
     on a.customer = b.store_id
     and a.bic_zdistr_ch = b.distribution_channel_id_cust
 left join hierarchies_product c
     on a.material = c.sku_id
     and a.bic_zdistr_ch = c.distribution_channel_id_prod
) g
group by g.date0, g.fiscper, g.promoted_product_group, g.promoted_product_group_desc, g.product_family_desc, g.product_segment_name, g.product_category_name, g.management_grouping_desc, g.state, g.ac_scbm_id, g.ac_scbm_desc, g.bic_zdistr_ch, g.state
"""
combined_raw_sku = spark.sql(query)
combined_raw_sku = combined_raw_sku.withColumn('date', next_day(to_date(col("date0"), "yyyyMMdd"), "saturday"))

#z.show(combined_raw_sku)

# %pyspark
# Aggregate daily data to weekly
agg_raw = combined_raw_sku.groupBy("date", "promoted_product_group", "promoted_product_group_desc", 
	"product_family_desc", "product_segment_name", "product_category_name", "management_grouping_desc",
	"state", "ac_scbm_id", "ac_scbm_desc", "bic_zdistr_ch").\
agg({'gross_sales':'sum','sales_pounds':'sum'}).\
withColumnRenamed("sum(gross_sales)","gross_sales").\
withColumnRenamed("sum(sales_pounds)","sales_pounds")

agg_raw = agg_raw.cache()

print(agg_raw.count())
z.show(agg_raw)

# %pyspark
# WIP - Write out Aggregated raw table for query / QC
agg_raw.createOrReplaceTempView("raw_ship_agg")
spark.sql("drop table if exists default.raw_ship_agg")
spark.sql("create table if not exists default.raw_ship_agg as select * from raw_ship_agg")

# %pyspark
# agg_raw_week = agg_raw_nodup_ppg_hier.withColumn("weekOfYear", weekofyear(col("date")))
agg_raw_week = agg_raw.withColumn("weekOfYear", weekofyear(col("date")))
agg_raw_week = agg_raw_week.withColumn("year", year(col("date")))

agg_raw_week_curr = agg_raw_week.withColumn("year", expr("year - 1"))

agg_raw_week_prev = agg_raw_week.select("year", "weekOfYear", 'promoted_product_group', "promoted_product_group_desc", 
	"product_family_desc", "product_segment_name", "product_category_name", "management_grouping_desc", "state", "ac_scbm_id", "ac_scbm_desc", "bic_zdistr_ch", "sales_pounds", "gross_sales").\
withColumnRenamed("sales_pounds", "stly_sales_pounds").withColumnRenamed("gross_sales", "stly_gross_sales")

# Fill nulls with NAs to enable correct join in Spark
agg_raw_week_curr = agg_raw_week_curr.fillna("Null_Value", subset=['promoted_product_group',"promoted_product_group_desc", 
	"product_family_desc", "product_segment_name", "product_category_name", "management_grouping_desc",'state','ac_scbm_id','ac_scbm_desc','bic_zdistr_ch'])
	
agg_raw_week_prev = agg_raw_week_prev.fillna("Null_Value", subset=['promoted_product_group',"promoted_product_group_desc", 
	"product_family_desc", "product_segment_name", "product_category_name", "management_grouping_desc",'state','ac_scbm_id','ac_scbm_desc','bic_zdistr_ch'])

# NOTE: Dataset has blank weekending dates but correct weekOfYear number. Use future dates (beyond curr week) to create a weekOfYear to weekending mapping file and retrieve the weekending.
agg_week = agg_raw_week_curr.join(agg_raw_week_prev, on = ['year','weekOfYear','promoted_product_group',"promoted_product_group_desc", 
	"product_family_desc", "product_segment_name", "product_category_name", "management_grouping_desc",'state','ac_scbm_id','ac_scbm_desc','bic_zdistr_ch'], how = 'outer')
z.show(agg_week.filter("year >= 2019"))

# %pyspark
week_to_we_map = agg_raw_week_curr.filter("year = 2019").select("weekOfYear","date").distinct()
z.show(week_to_we_map)

# %pyspark
# agg_week_2019_only = agg_raw_week_prev.join(agg_raw_week_curr, on = ['year','weekOfYear','promoted_product_group',"promoted_product_group_desc", 
# 	"product_family_desc", "product_segment_name", "product_category_name", "management_grouping_desc",'state','ac_scbm_id','ac_scbm_desc','bic_zdistr_ch'], how = 'left_anti')
# z.show(agg_week_2019_only.filter("year >= 2019"))
# agg_week_2019_only.write.format('parquet').mode('overwrite').option('header','true').save('/user/bwn2456/CBDA/ship_agg_week_2019_only/')

# %pyspark
# Channel mapping file
from pyspark.sql.types import *
channel_mapping = spark.createDataFrame(
[
("01", "Distribution Channel 01"),
("10", "Other"),
("11", "DSD Bis Intercompany"),
("12", "DSD Pizza Intercomp"),
("20", """Warehouse/Exports"""),
("30", "Foodservice"),
("40", "DSD Pizza"),
("45", "DSD"),
("50", "KFI"),
("55", "Plant Ingredient"),
("60", "Imports"),
("65", "Bulk FS - Specialty"),
],
StructType([StructField('bic_zdistr_ch',StringType(), True), StructField('channel_desc',StringType(), True)]) # add your columns label here
)
z.show(channel_mapping)

# %pyspark
agg_raw_chn = agg_week.join(channel_mapping, how='left', on='bic_zdistr_ch')

# %pyspark
# Write out
agg_out = agg_raw_chn.filter("year = 2019").filter("management_grouping_desc IS NOT NULL")

agg_out.createOrReplaceTempView("agg_out") 
spark.sql("DROP TABLE IF EXISTS default.cbda_ship")
spark.sql("create table if not exists default.cbda_ship as select * from agg_out")

# %pyspark
agg_model_cols = ['week_ending_date', 'channel', 'retailer', 'state', 'mdlz_business', 'mdlz_category', 'mdlz_brand', 'mdlz_ppg', 'promoted_product_group_desc', 'product_segment_name', 'retailer_desc', 'pos_qty', 'pos_dollar', 'week_of_year', 'year']

agg_model = agg_raw_chn.filter("management_grouping_desc IS NOT NULL")
agg_model = agg_model.withColumn("year", year(col("date")))

agg_model = agg_model.withColumnRenamed("date","week_ending_date").withColumnRenamed("ac_scbm_id","retailer").withColumnRenamed("management_grouping_desc","mdlz_business").withColumnRenamed("product_category_name","mdlz_category").withColumnRenamed("product_family_desc","mdlz_brand").withColumnRenamed("promoted_product_group","mdlz_ppg").withColumnRenamed("sales_pounds","pos_qty").withColumnRenamed("gross_sales","pos_dollar").withColumnRenamed("weekOfYear","week_of_year").withColumnRenamed("bic_zdistr_ch","channel").withColumnRenamed("ac_scbm_desc", "retailer_desc").select(*agg_model_cols)

z.show(agg_model)

# %pyspark
agg_model_all_cols = agg_raw_chn.withColumnRenamed("date","week_ending_date").withColumnRenamed("ac_scbm_id","retailer").withColumnRenamed("management_grouping_desc","mdlz_business").withColumnRenamed("product_category_name","mdlz_category").withColumnRenamed("product_family_desc","mdlz_brand").withColumnRenamed("promoted_product_group","mdlz_ppg").withColumnRenamed("sales_pounds","pos_qty").withColumnRenamed("gross_sales","pos_dollar").withColumnRenamed("weekOfYear","week_of_year").withColumnRenamed("bic_zdistr_ch","channel")
z.show(agg_model_all_cols.filter("retailer = 'US3000422' AND state = 'CA' AND channel = '20' AND mdlz_ppg = 'FS8'"))

# %pyspark
print(agg_model.count())
print(agg_model.select("retailer","state","mdlz_business","mdlz_category","mdlz_brand","mdlz_ppg","channel").distinct().count())

# %pyspark
# Write out
agg_model.createOrReplaceTempView("agg_model")
spark.sql("drop table if exists default.cbda_ship_model_input")
spark.sql("create table default.cbda_ship_model_input as select * from agg_model")

# %pyspark